# Discrete Dynamical Systems with Python!

Dynamical systems are rules describing the evolution of some variable $\vec{u}$ in time, according to some equations of motion given by a function $\vec{f}$.  
Here, we'll investigate _discrete dynamical systems_, often called _maps_, whereby time is discrete (in 'time-steps') and the equations of motion take the general form:

$$
\vec{u}_{n+1} = \vec{f}(\vec{u}_n,p,n)
$$

We'll use `numba`'s just-in-time compiler to compare our results with julia. Specifically, we'll try and use the `nopython` mode via the `njit` decorator, to ensure we make no (costly) calls to the Python interpreter.

In [ ]:
import numpy as np
from numba import njit

### De Jong attractor
Let's have a look at the following attractor:
$$
\begin{align}
x_{n+1} &= sin(a y_n) - cos(b x_n) \\
y_{n+1} &= sin(c x_n) - cos(d y_n)
\end{align}
$$

we define a function to perform this map, and compile it down:

In [ ]:
@njit
def dejong_eom(state, args):
    # unpack system parameters
    a, b, c, d = args
    
    # unpack the state vector
    x, y = state
    
    # map to the new state
    return np.array([np.sin(a*y) - np.cos(b*x),
                     np.sin(c*x) - np.cos(d*y)])

Next we define our initial state and parameters tuple

In [ ]:
state = np.array([-0.3, 0.2])
args = (-2.0, -2.0, -1.2, 2.0)
# evaluate dejong attractor once on the initial state
dejong_eom(state, args)

We then define our _higher order function_ to perfom the iteration. Note we're already using an in-place version

In [ ]:
@njit
def calc_orbit(out, fmap, x0, args):
    ''' Iterate the function fmap through a transient of warmup iterations and return an array of the final orbit.
        Inputs:
            out  - output array to store final orbit
            fmap - function of the iterative map
            x0   - initial value
            args - additional arguments taken by fmap
    '''
    out[0,:] = x0
    for i in range(len(out)-1):
        out[i+1,:] = fmap(out[i,:], args)

We can then time this for 10,000,000 iterations as before:

In [ ]:
%%timeit
N = int(10e6)
x0 = np.array([-0.3, 0.2])
args = (-2.0, -2.0, -1.2, 2.0)
out = np.zeros((N, len(x0)))

calc_orbit(out, dejong_eom, x0, args)

### Post-processing binning
We post-process the points, by binning them according to visitation density.

In [ ]:
@njit
def bin_data(out, z, n):
    ''' Map a continuous coordinate to a discrete point on a grid of length n.
        Inputs:
            out - output array of pixels
            z - array of coordinate values
            n - length of grid
    '''
    out[:] = (z - z.min())/(z.max() - z.min())*(n-1)

In [ ]:
nx, ny = 801, 801
bins = np.empty(out.shape, dtype=np.int64)
bin_data(bins[:,0], out[:,0], nx)
bin_data(bins[:,1], out[:,1], ny)

In [ ]:
def get_counts(bins):
    ''' Compute number of occurrences of each binned coordinate value.
        Inputs:
            bins - array of visited pixels
        Outputs:
            counts - logarithm of counts
    '''
    inverse, counts = np.unique(bins[:,0] + bins[:,1]*1j, return_counts=True, return_inverse=True)[1:]
    counts = np.log2(1 + counts[inverse])
    return counts

counts = get_counts(bins)

In [ ]:
@njit
def make_map(image, bins, counts):
    ''' Populate image pixels with corresponding counts.
        Inputs:
            image - two-dimensional image array
            bins -  array of visited pixels
            counts - array of counts for each pixel
    '''
    for i in range(len(bins)):
        image[bins[i,0], bins[i,1]] = counts[i]

Putting it all together in a function:

In [ ]:
def make_attractor(image, fmap, s0, args, N):
    ''' Make an image colored by each point's visitation density.
        Inputs:
            image - output image
            fmap - function of the iterative map
            x0 - initial value
            args - additional arguments taken by fmap
            N -  total number of iterations
    '''
    # compute orbit
    out = np.zeros((N, len(s0)))
    calc_orbit(out, fmap, s0, args)

    # format data to grid
    nx, ny = image.shape
    bins = np.zeros(out.shape, dtype=np.int64)
    bin_data(bins[:,0], out[:,0], nx)
    bin_data(bins[:,1], out[:,1], ny)

    # populate image by visitation density
    counts = get_counts(bins)
    make_map(image, bins, counts)

In [ ]:
nx, ny = 801, 801
image = np.zeros((nx, ny), dtype=np.int64)

N = int(10e6)
s0 = np.array([-0.3, 0.2])
args = (-2., -2., -1.2, 2.)
make_attractor(image, dejong_eom, s0, args, N)

### Visualization
Finally, we can plot the visitation density bins of an attractor as an image

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['axes.linewidth'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['mathtext.default'] = 'regular'

def plot_attractor(image, palette='inferno'):
    ''' Plot the attractor image.
        Inputs:
          image - two-dimensional image array
          palette - colormap name
        Outputs:
          f, ax - figure and axis objects of resulting plot
    '''
    # set up figure and axes
    f = plt.figure(figsize=(9,9))
    ax = f.add_subplot()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # create a border by padding with some zeros
    npad = int(0.1*image.shape[0])
    image = np.pad(image, ((npad,npad), (npad,npad)), mode='constant')

    # display image
    ax.imshow(np.flipud(image), cmap=palette)
    return f, ax

In [ ]:
f, ax = plot_attractor(image, palette='inferno');